In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings("ignore", message="The structure of `inputs` doesn't match the expected structure")
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Concatenate, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.optimizers import Adam

In [36]:
df = pd.read_csv('data_output/charting_clean.csv', low_memory=False)

In [37]:
df.head()

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,2nd_test,rallyCountVerification,Player_svr,Player_ret,1st_player_svr,1st_player_ret,2nd_player_svr,2nd_player_ret,1st_final,2nd_final
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0,1 (1),1,0,...,NaN,1,Novak Djokovic,Ben Shelton,6,f2n#,NaN,NaN,6 f2n#,NaN
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0,1 (2),1,0,...,6 b19 f1 b2 s1 f3 f2 j2 *,8,Novak Djokovic,Ben Shelton,4n,NaN,6 f1 s1 f2,b19 b2 f3 j2*,4n,6 b19 f1 b2 s1 f3 f2 j2*
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0.0,0.0,15-15,1 (3),1,0,...,4 b28 f2 o1 *,4,Novak Djokovic,Ben Shelton,4d,NaN,4 f2,b28 o1*,4d,4 b28 f2 o1*
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0.0,0.0,15-30,1 (4),1,0,...,NaN,3,Novak Djokovic,Ben Shelton,6 f3*,s28,NaN,NaN,6 s28 f3*,NaN
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0.0,0.0,30-30,1 (5),1,0,...,NaN,3,Novak Djokovic,Ben Shelton,4 b3*,b37,NaN,NaN,4 b37 b3*,NaN


# Modelo de predicción de final de punto
El objetivo de este modelo es predecir el final de un punto en un partido de tenis, es decir, si el punto terminará con un golpe ganador, un error forzado o un error no forzado.

In [38]:
# Select relevant columns
point_winning_data = df[['1stIn', '2ndIn', '1st_final', '2nd_final', 'Pt', 'Set1', 'Set2', 'Gm1', 'Gm2', 'TbSet', 'TB?', 'TBpt', 'Svr', 'Ret', 'isAce', 'isUnret', 'isRallyWinner', 'isForced', 'isUnforced', 'isDouble', 'PtWinner', 'isSvrWinner', 'rallyCount']]

# Replace values in '1st_final' with '2nd_final' if '1stIn' == 0
point_winning_data['1st_final'] = np.where(point_winning_data['1stIn'] == 0, point_winning_data['2nd_final'], point_winning_data['1st_final'])

# Drop rows where '2ndIn' is 0
point_winning_data = point_winning_data[point_winning_data['2ndIn'] != 0]

# Display the first few rows of the DataFrame
point_winning_data.head()

,1stIn,2ndIn,1st_final,2nd_final,Pt,Set1,Set2,Gm1,Gm2,TbSet,...,Ret,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount
0,1,NaN,6 f2n#,NaN,1,0,0,0.0,0.0,1,...,2,False,False,False,True,False,False,1,1,1
1,0,1.0,6 b19 f1 b2 s1 f3 f2 j2*,6 b19 f1 b2 s1 f3 f2 j2*,2,0,0,0.0,0.0,1,...,2,False,False,True,False,False,False,2,0,8
2,0,1.0,4 b28 f2 o1*,4 b28 f2 o1*,3,0,0,0.0,0.0,1,...,2,False,False,True,False,False,False,2,0,4
3,1,NaN,6 s28 f3*,NaN,4,0,0,0.0,0.0,1,...,2,False,False,True,False,False,False,1,1,3
4,1,NaN,4 b37 b3*,NaN,5,0,0,0.0,0.0,1,...,2,False,False,True,False,False,False,1,1,3


In [39]:
#drop na if na in 1st_final
point_winning_data = point_winning_data.dropna(subset=['1st_final'])

In [40]:
#point_winning_data.to_csv('data_output/full_sequence.csv', index=False)

In [41]:
point_winning_data = point_winning_data.sample(frac=0.5, random_state=42) # 50% de los datos

# Preprocesamiento de datos

In [42]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(point_winning_data['1st_final'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Convertir las secuencias en listas de tokens
tokenized_sequences = []
for seq in point_winning_data['1st_final']:
    tokens = seq.split()
    tokenized_sequence = tokenizer.texts_to_sequences([tokens])
    tokenized_sequence = [item for sublist in tokenized_sequence for item in sublist]
    tokenized_sequences.append(tokenized_sequence)

# Rellenar secuencias para que todas tengan la misma longitud
max_sequence_length = 10  # Máximo de 5 golpes para unificar el largo de las secuencias
X_seq = pad_sequences(tokenized_sequences, maxlen=max_sequence_length, padding='post')


In [43]:
point_winning_data['TbSet'] = point_winning_data['TbSet'].replace('T',2).astype(float)
point_winning_data['TB?'] = point_winning_data['TbSet'].replace('S',2).astype(float)

In [44]:
point_winning_data.head()

,1stIn,2ndIn,1st_final,2nd_final,Pt,Set1,Set2,Gm1,Gm2,TbSet,...,Ret,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount
592200,0,1.0,5 b38 s2n@,5 b38 s2n@,149,2,0,0.0,1.0,1.0,...,2,False,False,False,False,True,False,2,0,2
484980,1,NaN,6 f2n#,NaN,101,1,0,5.0,4.0,1.0,...,2,False,False,False,True,False,False,1,1,1
411659,1,NaN,4 b38 b2 f1 f1 f3n@,NaN,109,1,0,0.0,0.0,1.0,...,1,False,False,False,False,True,False,2,1,5
570119,0,1.0,5 b28 b3w@,5 b28 b3w@,136,2,0,4.0,5.0,1.0,...,2,False,False,False,False,True,False,2,0,2
159868,1,NaN,6 b29 b2 b2 f2 b2 f2 b2 f19w@,NaN,198,2,1,0.0,0.0,1.0,...,1,False,False,False,False,True,False,1,0,8


In [45]:
from sklearn.preprocessing import StandardScaler

X_context_features = point_winning_data[['Pt', 'Set1', 'Set2', 'Gm1', 'Gm2', 'TbSet', 'TB?', 'TBpt', 'Svr', 'Ret', 'isAce', 'isUnret', 'isRallyWinner', 'isForced', 'isUnforced', 'isDouble', 'PtWinner', 'isSvrWinner', 'rallyCount']]

scaler = StandardScaler()
context_features = scaler.fit_transform(X_context_features)

print(f"X_seq shape: {X_seq.shape}, context_features shape: {X_context_features.shape}")

X_seq shape: (356652, 10), context_features shape: (356652, 19)


In [46]:
# definir las etiquetas y, 0 si fue winner (*), 1 si fue error forzado (#), 2 si fue error no forzado(@)
point_winning_data.head()
#Si ultimo caracter de 1st_final es *, entonces 0, si es # entonces 1, si es @ entonces 2
y_type = []
for i in point_winning_data['1st_final']:
    if i[-1] == '*':
        y_type.append(0)
    elif i[-1] == '#':
        y_type.append(1)
    else:
        y_type.append(2)
        
print(f'X_seq shape: {X_seq.shape}, context_features shape: {X_context_features.shape}, y_type shape: {len(y_type)}')

X_seq shape: (356652, 10), context_features shape: (356652, 19), y_type shape: 356652


In [47]:
seq_input = Input(shape=(max_sequence_length,))
context_input = Input(shape=(X_context_features.shape[1],))

# Procesamiento secuencial con Embedding y LSTM
embedding_layer = Embedding(input_dim=vocab_size, output_dim=64)(seq_input)
lstm_layer = LSTM(64)(embedding_layer)

# Procesamiento de características contextuales con capas densas
dense_context = Dense(32, activation='relu')(context_input)

# Combinar ambas entradas
merged = Concatenate()([lstm_layer, dense_context])
dropout_layer = Dropout(0.2)(merged)
output = Dense(3, activation='softmax')(dropout_layer)  # 3 clases: ganador, error forzado, error no forzado

# Definir y compilar el modelo
model = Model(inputs=[seq_input, context_input], outputs=output)
model.compile(optimizer=Adam(learning_rate= 0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()

# Etiquetas de entrenamiento (por simplicidad, generamos algunas etiquetas para el ejemplo)
y_type = np.array(y_type)  # Ejemplo: 0 - ganador, 1 - error forzado, 2 - error no forzado

# Entrenar el modelo
history = model.fit(
    [X_seq, X_context_features], y_type,
    epochs=10,  # Reducido para una prueba rápida
    batch_size=64,
    validation_split=0.2
)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 10, 64)    │     43,008 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 64)        │     33,024 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │        640 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 96)        │          0 │ lstm_4[0][0],     │
│ (Concatenate)       │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 96)        │          0 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │        291 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 76,963 (300.64 KB)

 Trainable params: 76,963 (300.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.4921 - loss: 0.9692 - val_accuracy: 0.5833 - val_loss: 0.8663
Epoch 2/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.5870 - loss: 0.8639 - val_accuracy: 0.5966 - val_loss: 0.8471
Epoch 3/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5995 - loss: 0.8462 - val_accuracy: 0.6065 - val_loss: 0.8367
Epoch 4/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.6071 - loss: 0.8349 - val_accuracy: 0.6079 - val_loss: 0.8274
Epoch 5/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.6101 - loss: 0.8263 - val_accuracy: 0.6108 - val_loss: 0.8209
Epoch 6/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.6138 - loss: 0.8204 - val_accuracy: 0.6153 - val_loss: 0.8149
Epoch 7/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.6162 - loss: 0.8138 - val_accuracy: 0.6188 - val_loss: 0.8112
Epoch 8/10
4459/4459 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.6201 - loss: 0

# Prediccion

In [48]:
# Secuencia y características contextuales de ejemplo
new_sequence = "6 f2 f1 f3 o1 s2"  # Ejemplo de secuencia de golpes
new_sequence_tokenized = tokenizer.texts_to_sequences([new_sequence.split()])
new_sequence_padded = pad_sequences(new_sequence_tokenized, maxlen=max_sequence_length, padding='post')

# Nuevas características contextuales (tomando la primera muestra del contexto de prueba)
test_context_features = context_features[0]  # X_context contiene las características normalizadas del conjunto de datos original

# Transformar la nueva característica a una matriz con una fila
test_context_features = test_context_features.reshape(1, -1)

# Hacer la predicción
predicted_probabilities = model.predict([new_sequence_padded, test_context_features])
predicted_class = np.argmax(predicted_probabilities)

# Interpretar la predicción
ending_types = {0: "Winner (*)", 1: "Forced Error (#)", 2: "Unforced Error (@)"}
print(f"Predicción: {ending_types[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Predicción: Winner (*)


In [60]:
# Secuencia y características contextuales de ejemplo
new_sequence = ("f1 b2 o1")  # Ejemplo de secuencia de golpes
new_sequence_tokenized = tokenizer.texts_to_sequences([new_sequence.split()])
new_sequence_padded = pad_sequences(new_sequence_tokenized, maxlen=max_sequence_length, padding='post')

# Nuevas características contextuales (tomando la primera muestra del contexto de prueba)
test_context_features = context_features[42]  # X_context contiene las características normalizadas del conjunto de datos original

# Transformar la nueva característica a una matriz con una fila
test_context_features = test_context_features.reshape(1, -1)

# Hacer la predicción
predicted_probabilities = model.predict([new_sequence_padded, test_context_features])[0]

# Interpretar la predicción y mostrar las probabilidades
ending_types = {0: "Winner (*)", 1: "Forced Error (#)", 2: "Unforced Error (@)"}
print("Probabilidades de cada clase:")
for i, probability in enumerate(predicted_probabilities):
    print(f"{ending_types[i]}: {probability:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Probabilidades de cada clase:
Winner (*): 0.1554
Forced Error (#): 0.7373
Unforced Error (@): 0.1073


**Interpretación**
Tiene sentido que luego de un golpe de fondo (b2) y un smash (o1) el siguiente sea un error forzado (#). Las probabilidades de meter un winner luego de un smash es muy baja, y es dificil considerar un error no forzado luego de un smash.